In [3]:
#1.1

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

traindata = pd.read_csv("hw4Train.csv")
testdata = pd.read_csv("hw4Test.csv")
print(testdata.head())
print(traindata.head())

X = traindata.iloc[:, :-1]  # Features
y = traindata.iloc[:, -1]   # Target

Xtest = testdata.iloc[:, :-1]
ytest = testdata.iloc[:, -1]

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.2, random_state=42)

maxdepth = 10 
predefinedAcc = []


#Tree with no max depth
print("No max depth")
tree0 = DecisionTreeClassifier(random_state=42)
tree0.fit(Xtrain, ytrain)
testpred0 = tree0.predict(Xtest) 
accuracytest0 = accuracy_score(ytest, testpred0)
print(f'Accuracy on test set no max depth: {accuracytest0:.8f}')
print('Classification Report:')
print(classification_report(ytest, testpred0, zero_division=1))
predefinedAcc.append(accuracytest0)

#Searching for best depth
def depthsearch(Xtrain, ytrain, Xval, yval):
    best_depth = None
    best_accuracy = 0
    for depth in range(1, 25):  # Trying depths from 1 to 50
        tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
        tree.fit(Xtrain, ytrain)
        valpred = tree.predict(Xval)
        accuracy = accuracy_score(yval, valpred)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
    return best_depth


# Find the best max_depth using the validation set
best_max_depth = depthsearch(Xtrain, ytrain, Xval, yval)
print(f'Best max_depth found: {best_max_depth}')
tree2 = DecisionTreeClassifier(max_depth=best_max_depth, random_state=42)
tree2.fit(Xtrain, ytrain)
testpred = tree2.predict(Xtest)
accuracy = accuracy_score(ytest, testpred)
print(f'Accuracy on test set with optimal depth: {accuracy:.8f}')
print('Classification Report:')
print(classification_report(ytest, testpred, zero_division=1))





   flow_duration  Header_Length  Protocol Type   Duration      Rate     Srate  \
0      -0.008535      -0.166991      -0.343010  -0.166831 -0.092989 -0.092989   
1      -0.018286      -0.137768       0.844474   0.105423 -0.025106 -0.025106   
2      -0.018563      -0.167364      -0.904734  13.445896 -0.092626 -0.092626   
3      -0.014154      -0.167061      -0.343010  -0.166831 -0.092970 -0.092970   
4      -0.018563      -0.167247      -0.343010  -0.166831 -0.092983 -0.092983   

      Drate  fin_flag_number  syn_flag_number  rst_flag_number  ...       Std  \
0 -0.003358        -0.307779         1.962705        -0.315717  ... -0.205656   
1 -0.003358        -0.307779        -0.509501        -0.315717  ... -0.201416   
2 -0.003358        -0.307779        -0.509501        -0.315717  ... -0.205656   
3 -0.003358        -0.307779         1.962705        -0.315717  ... -0.205656   
4 -0.003358        -0.307779        -0.509501        -0.315717  ... -0.205656   

   Tot size       IAT    N

Code for attempting to combine multiple optimal features, however using optimal max depth found earlier with optimal feature count resulted in lower accuracy, indicating the the relationship between the hyperparameters is not linear.


In [4]:
#Searching for best feature count, not used
#Source:
#https://scikit-learn.org/stable/modules/tree.html
def featuresearch(Xtrain, ytrain, Xval, yval):
    best_featurecount = None
    best_accuracy = 0
    for featurecount in range(1, 50):  # Trying features from 1 to 50
        tree = DecisionTreeClassifier(max_depth=best_max_depth, random_state=42, max_features=featurecount)
        tree.fit(Xtrain, ytrain)
        valpred = tree.predict(Xval)
        accuracy = accuracy_score(yval, valpred)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_featurecount = featurecount
    return best_featurecount


#best_featurecount_found = featuresearch(Xtrain, ytrain, Xval, yval)
#print(f'Best feature count found: {best_featurecount_found}')
#tree4 = DecisionTreeClassifier(max_depth=best_max_depth, random_state=42, max_features=best_featurecount_found)
#tree4.fit(Xtrain, ytrain)
#testpred = tree4.predict(Xtest)
#accuracy = accuracy_score(ytest, testpred)
#print(f'Accuracy on test set with optimal featurecount: {accuracy:.4f}')
#print('Classification Report:')
#print(classification_report(ytest, testpred, zero_division=1))

#def leafsearch(Xtrain, ytrain, Xval, yval):
#    best_featurecount = None
#    best_accuracy = 0
#    for featurecount in range(1, 20):  # 
#        tree = DecisionTreeClassifier(max_depth=best_max_depth, random_state=42, max_features=featurecount)
#        tree.fit(Xtrain, ytrain)
#        valpred = tree.predict(Xval)
#        accuracy = accuracy_score(yval, valpred)
#        if accuracy > best_accuracy:
#            best_accuracy = accuracy
#            best_featurecount = featurecount
#    return best_featurecount

The optimal depth for the decision tree classifier, however the differences in accuracy were very small. This was also true for the F1-score where the optimal parameters with macro avg was 0.82.

SVM may not be optimal for this dataset due to the high number of features that would require kernel tuning. 



Feature selection is a vital step in the machine learning workflow that enhances model performance, reduces complexity, and improves interpretability. By identifying and retaining relevant features while eliminating redundant and irrelevant ones, we can achieve more accurate and useful models.

In [6]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.preprocessing import MinMaxScaler
#Source:
#https://scikit-learn.org/stable/modules/feature_selection.html

# Load the dataset
traindata = pd.read_csv("hw4Train.csv")
testdata = pd.read_csv("hw4Test.csv")
#print(testdata.head())
#print(traindata.head())

X = traindata.iloc[:, :-1]  # All columns except the last
y = traindata.iloc[:, -1]   # The last column


feature_names = X.columns

# Scale features to a range [0,1] for chi-square calculation
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Information Gain (using mutual information)
info_gain = mutual_info_classif(X, y)
info_gain_series = pd.Series(info_gain, index=feature_names)
info_gain_ranking = info_gain_series.sort_values(ascending=False)

# Chi-square test
chi2_values, p_values = chi2(X_scaled, y)
chi2_series = pd.Series(chi2_values, index=feature_names)
chi2_ranking = chi2_series.sort_values(ascending=False)

# Display the rankings
print("Feature ranking based on Information Gain:")
print(info_gain_ranking)

print("\nFeature ranking based on Chi-square:")
print(chi2_ranking)

   flow_duration  Header_Length  Protocol Type   Duration      Rate     Srate  \
0      -0.008535      -0.166991      -0.343010  -0.166831 -0.092989 -0.092989   
1      -0.018286      -0.137768       0.844474   0.105423 -0.025106 -0.025106   
2      -0.018563      -0.167364      -0.904734  13.445896 -0.092626 -0.092626   
3      -0.014154      -0.167061      -0.343010  -0.166831 -0.092970 -0.092970   
4      -0.018563      -0.167247      -0.343010  -0.166831 -0.092983 -0.092983   

      Drate  fin_flag_number  syn_flag_number  rst_flag_number  ...       Std  \
0 -0.003358        -0.307779         1.962705        -0.315717  ... -0.205656   
1 -0.003358        -0.307779        -0.509501        -0.315717  ... -0.201416   
2 -0.003358        -0.307779        -0.509501        -0.315717  ... -0.205656   
3 -0.003358        -0.307779         1.962705        -0.315717  ... -0.205656   
4 -0.003358        -0.307779        -0.509501        -0.315717  ... -0.205656   

   Tot size       IAT    N